## 에이전트의 Observability 활성화

이 과정에서는 에이전트에서 OpenTelemetry를 빠르게 활성화하여 에이전트와의 상호 작용이 자동으로 기록되고 내보내지도록 하는 방법을 보여줍니다.

필요한 모듈 설치

In [ ]:
pip install opentelemetry-exporter-otlp-proto-grpc

#### OpenTelemetry 활성화 방법
관찰가능성을 활성화하는 가장 간단한 방법은 configure_otel_providers을 사용하는 것입니다.

In [7]:
from agent_framework.observability import configure_otel_providers

# Reads OTEL_EXPORTER_OTLP_* environment variables automatically
configure_otel_providers()

### Console을 통한 Trace 확인

가장 기본적인 방법으로 Console에 trace가 나오도록 설정하고 에이전트를 실행해 보겠습니다.

In [ ]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from agent_framework.observability import configure_otel_providers
from opentelemetry.sdk.trace.export import ConsoleSpanExporter
from opentelemetry.sdk._logs.export import ConsoleLogRecordExporter 

# Console에 trace와 log를 출력하도록 설정
configure_otel_providers(
    exporters=[
        ConsoleSpanExporter(),        # Trace를 콘솔에 출력
        ConsoleLogRecordExporter()    # Log를 콘솔에 출력
    ]
)

# Create the agent - telemetry is automatically enabled
agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
    name="Joker",
    instructions="You are good at telling jokes who responds in korean."
)

# Run the agent
result = await agent.run("Tell me a joke about a pirate.")
print(result.text)

### 중요 ###
# 위의 코드를 Jyputer Notebook에서 실행하는 경우에는 Console에 Trace가 출력되지 않을 수 있습니다.
# 이 경우에는 Observability.py 파일을 별로도 만들어서 테스트 해야 합니다. 
# 이미 현재 폴더에는 완성된 Observability.py 파일이 있으므로, 해당 파일을 열어서 코드를 확인해 보고 실행합니다.
# 실행을 위해서는 우선 터미널을 열고, 가상환경을 활성화 한 후에 다음 명령어를 실행합니다.
#
# .\.venv\Scripts\Activate.ps1; 
# python .\MAF\Observability.py

Console에서 ./MAF/Observability.py 파일을 실행하면 결과는 다음과 같습니다.

![console](images/maf-trace-console.png)

### Aspire Dashboard를 사용한 Trace 시각화

로컬에서 Trace를 확인하기 위해서는 뛰어난 텔레메트리 UI를 제공하는 Aspire 대시보드를 사용할 수 있습니다. 이는 Docker를 사용해서 로컬에서 실행됩니다.

**도커를 사용해서 Aspire Dashboard 설치하기**

만일, 로컬 머신에 Docker가 설치되어 있지 않다면 [Docker 설치 가이드](https://docs.docker.com/get-docker/)를 참고하여 설치하시기 바랍니다. 만일, Docker를 설치하기에 어려움이 있다면 이번 실습은 건너뛰셔도 무방합니다. 동료의 진행상황을 함께 확인하시거나, 코치의 데모를 참고하시기 바랍니다. 

```Bash
# Pull and run the Aspire Dashboard container
docker run --rm -it -d \
    -p 18888:18888 \
    -p 4317:18889 \
    --name aspire-dashboard \
    mcr.microsoft.com/dotnet/aspire-dashboard:latest
```

이 명령은 다음과 같이 대시보드를 시작합니다:

Web UI: http://localhost:18888 로 접근하면 됩니다.  
OTLP endpoint: http://localhost:4317 로 애플리케이션이 텔레메트리 데이터를 전송할 수 있습니다.

이에 따라, 다음과 같은 환경변수를 .env 파일에 추가합니다.   

```Bash
ENABLE_INSTRUMENTATION=true
OTEL_EXPORTER_OTLP_ENDPOINT=http://localhost:4317
```

환경변수를 추가했다면, 환경변수를 다시 로드합니다.

In [12]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일 로드

True

### 에이전트 생성 및 실행

이제 에이전트를 실행하여 trace가 잘 기록되는지 확인합니다.

In [ ]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from agent_framework.observability import configure_otel_providers
from opentelemetry.sdk.trace.export import ConsoleSpanExporter
from opentelemetry.sdk._logs.export import ConsoleLogRecordExporter 

# 환경변수에 지정된 OTEL_EXPORTER_OTLP_ENDPOINT로 전송됨 (Aspire Dashboard)
configure_otel_providers()

# Create the agent - telemetry is automatically enabled
agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
    name="Joker",
    instructions="You are good at telling jokes who responds in korean."
)

# Run the agent
result = await agent.run("Tell me a joke about a pirate.")
print(result.text)

# Trace와 Log는 자동으로 콘솔(OUTPUT 패널)에 출력됩니다
# VS Code에서 "OUTPUT" 패널(Ctrl+Shift+U)을 열어 확인하세요

해적이 가장 좋아하는 알파벳은 뭔지 알아요?  
바로 "Rrrrr!" (아르르르!)


### Aspire Dashboard 에서 Trace 확인

http://localhost:18888 로 접속하여 Aspire Dashboard가 실행 중인지 확인합니다.  

로그인을 위해서 토큰을 입력하라고 나온다면, 다음과 같이 Docker Log에서 링크를 확인할 수 있습니다. 이 링크를 사용해서 로그인하거나, 로그인 화면에 토큰값으로 login?t= 이후의 값을 복사해서 입력합니다.

>Login to the dashboard at http://localhost:18888/login?t=******** . The URL may need changes depending on how network access......

![image](images/maf-docker2.png)

그러면, Aspire Dashboard UI가 나타납니다. 여기에서 추적[Trace] 탭을 클릭하여 에이전트와의 상호작용에서 생성된 Trace 데이터를 확인할 수 있습니다.

![Aspire Dashboard UI](images/maf-docker3.png)